# From RAGs to riches: Build an AI document interrogation app in 30 mins

PyData NYC 2023 | Dharhas Pothina

---

## Hi! I'm Dharhas 👋

- I'm from Quansight 

![caption](images/viking_boat_desk.jpg)

## Goal - Build this app

TODO: Demonstrate the final app for initial motivation

![caption](images/chatgpt_what_is_ragna.png)

## RAG - Make LLMs more useful

TODO: What is RAG?

## Tools

### Ragna

(TODO: Logo, what is Ragna?)

### Panel

(TODO: Logo, what is Panel?)

## Provide more data

In [1]:
document_path = "data/what_is_ragna.txt"

print(open(document_path, "r").read())

Ragna is a new open source project built by Quansight. It is designed to allow organizations to explore the power of Retrieval-augmented generation (RAG) based AI tools. Ragna provides an intuitive API for quick experimentation and built-in tools for creating production-ready applications allowing you to quickly leverage Large Language Models (LLMs) for your work.

At its core, Ragna is a plugin-based framework with a scalable queue based backend that provides:

 - Python API designed for experimentation that allows you to explore and test different LLMs, vector databases and embedding models quickly in Python.

- A REST API that allows you to build custom RAG-based web applications for your particular needs.

- A fully featured web application built with Panel (https://panel.holoviz.org) to select and configure LLMs, upload documents, and chat with the LLM. Designed for use as an out-of-the-box solution or as a reference to build custom web applications.

The Ragna website is https://

## Setup

In [2]:
from dotenv import load_dotenv
import ragna
from ragna import Config

print(f"ragna v{ragna.__version__}")

# Replace with your own OpenAI API KEY.
# This will be revoked after the talk
load_dotenv("api_keys")

config = Config()
print(config)

ragna v0.1.1.dev3+g223aa4c.d20231029235834
local_cache_root=PosixPath('/Users/dharhas/.cache/ragna') core=CoreConfig(queue_url='memory', document=<class 'ragna.core.LocalDocument'>, source_storages=[<class 'ragna.source_storages.RagnaDemoSourceStorage'>], assistants=[<class 'ragna.assistants.RagnaDemoAssistant'>]) api=ApiConfig(url='http://127.0.0.1:31476', database_url='memory', authentication=<class 'ragna.core.RagnaDemoAuthentication'>, upload_token_secret='u90AaTx8-QTRIlNsx-3Ghb65qjA0ycHm', upload_token_ttl=300) ui=UiConfig(url='http://127.0.0.1:31477')


## Ragna 101

In [3]:
from ragna.assistants import RagnaDemoAssistant
from ragna.core import Rag
from ragna.source_storages import RagnaDemoSourceStorage

In [4]:
rag = Rag(config)

async with rag.chat(
    documents=[document_path],
    source_storage=RagnaDemoSourceStorage,
    assistant=RagnaDemoAssistant,
) as chat:
    prompt = "What is Ragna?"
    answer = await chat.answer(prompt)

print(answer)

I can't really help you with your prompt:

> What is Ragna?

I can at least show you the sources that I was given:

- what_is_ragna.txt: Ragna is a new open source project built by Quansight. It is designed to allow organizations [...]


In [5]:
from ragna.assistants import Gpt4, Gpt35Turbo16k
from ragna.source_storages import Chroma, LanceDB

In [6]:
async with rag.chat(
    documents=[document_path],
    source_storage=RagnaDemoSourceStorage,
    assistant=Gpt35Turbo16k,
) as chat:
    prompt = "What is Ragna?"
    answer = await chat.answer(prompt)

print(answer)

Ragna is a new open source project developed by Quansight. It is designed to enable organizations to efficiently manage and scale their data science workflows. Ragna aims to provide a unified platform for data scientists to collaborate, share code, and deploy models. It offers features such as version control, reproducibility, and scalability to help streamline the data science process.


In [11]:
answer.sources[0].content

'Ragna is a new open source project built by Quansight. It is designed to allow organizations [...]'

In [12]:
async with rag.chat(
    documents=[document_path],
    source_storage=Chroma,
    assistant=Gpt35Turbo16k,
) as chat:
    prompt = "What is Ragna?"
    answer = await chat.answer(prompt)

In [14]:
print(answer)

Ragna is an open source project developed by Quansight. It is a plugin-based framework that allows organizations to explore the power of Retrieval-augmented generation (RAG) based AI tools. Ragna provides an intuitive Python API for experimentation with different Large Language Models (LLMs), vector databases, and embedding models. It also offers a REST API for building custom RAG-based web applications and a fully featured web application for selecting and configuring LLMs, uploading documents, and interacting with the LLM through chat. Ragna is designed to be used as an out-of-the-box solution or as a reference for building custom web applications. You can find more information about Ragna on its website at https://ragna.chat/ and its source code on GitHub at https://github.com/Quansight/ragna.


In [15]:
sec_10k_documents = [
    "files/ford-10k-2022.pdf",
    "files/gm-10k-2022.pdf",
    "files/tesla-10k-2022.pdf",
]

chat = rag.chat(documents=sec_10k_documents, 
                source_storage=Chroma,
                assistant=Gpt35Turbo16k,
               )

await chat.prepare()

Message(content='How can I help you with the documents?', role=<MessageRole.SYSTEM: 'system'>, sources=[])

![caption](images/rag_diagram.png)

In [43]:
answer = await chat.answer("Did Ford earn more than GM?")

In [44]:
print(answer.content)

Based on the information provided, Ford's net sales and revenue for the year ended December 31, 2021, were $128,378 million, while GM's net sales and revenue for the same period were $101,308 million. Therefore, Ford earned more than GM during that year.


In [45]:
answer = await chat.answer("Did Tesla earn more than GM?")

In [38]:
print(answer.content)

Based on the information provided, Tesla's total revenues for the year ended December 31, 2022 were $81,462 million. On the other hand, General Motors' total net sales and revenue for the same period were $156,735 million. Therefore, General Motors earned more than Tesla.


In [54]:
answer = await chat.answer("How much did Tesla, GM and Ford earn")

In [55]:
print(answer.content)

The information provided does not include the earnings of Tesla, GM, and Ford. The financial statements provided are for Tesla, Inc. and General Motors Company. To obtain the earnings of these companies, you would need to refer to their respective financial statements or annual reports.


In [59]:
import asyncio
import itertools
from pprint import pprint

from ragna.assistants import Gpt4, Gpt35Turbo16k
from ragna.source_storages import Chroma, LanceDB

source_storages = [Chroma, LanceDB]
assistants = [Gpt35Turbo16k, Gpt4]

prompt = "How much did Tesla, GM and Ford earn"

async def answer_prompt(source_storage, assistant):
    async with rag.chat(
        documents=sec_10k_documents,
        source_storage=source_storage,
        assistant=assistant,
    ) as chat:
        message = await chat.answer(prompt)
        return message.content

experiments = {
    (source_storage.display_name(), assistant.display_name()): answer_prompt(
        source_storage, assistant
    )
    for source_storage, assistant in itertools.product(source_storages, assistants)
}

pprint(experiments)

{('Chroma', 'OpenAI/gpt-3.5-turbo-16k'): <coroutine object answer_prompt at 0x16ea91840>,
 ('Chroma', 'OpenAI/gpt-4'): <coroutine object answer_prompt at 0x16ea91dc0>,
 ('LanceDB', 'OpenAI/gpt-3.5-turbo-16k'): <coroutine object answer_prompt at 0x16ea91bc0>,
 ('LanceDB', 'OpenAI/gpt-4'): <coroutine object answer_prompt at 0x16ea91e40>}


In [60]:
results = dict(zip(experiments.keys(), await asyncio.gather(*experiments.values())))
pprint(results)

{('Chroma', 'OpenAI/gpt-3.5-turbo-16k'): 'The information provided does not '
                                         'include the earnings of Tesla, GM, '
                                         'and Ford. The financial statements '
                                         'provided are for Tesla, Inc. and '
                                         'General Motors Company. To obtain '
                                         'the earnings of these companies, you '
                                         'would need to refer to their '
                                         'respective financial statements or '
                                         'annual reports.',
 ('Chroma', 'OpenAI/gpt-4'): 'The text provided does not include information '
                             'on the earnings of Tesla, GM, and Ford.',
 ('LanceDB', 'OpenAI/gpt-3.5-turbo-16k'): "I'm sorry, but I don't have access "
                                          'to real-time financial information. '
    

In [ ]:
# TODO: Asssitant, Source Storage, RAG, Chat, AsyncIO

In [ ]:
# TODO: REST API

## Panel 101

In [61]:
# TODO
import panel as pn

pn.extension()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Let's build the app!

In [ ]:
# TODO

## Thanks! Question?

Check out: **ragna.chat**

Share your feedback!